In [1]:
import numpy as np
import pandas as pd

import pandas_datareader as pdr

import matplotlib.pyplot as plt
import matplotlib.dates as mpl_dates
import mplfinance as mpf
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import chart_studio.plotly as py
import plotly.offline as pyo
import talib
from ortisan_ta import analysis, statistics

In [2]:
import requests
symbol = "WINQ22"
initial_date = "2022-08-01"
final_date = "2022-08-01"
r = requests.get(f"http://192.168.15.28:8000/prices?symbol={symbol}&timeframe=TIMEFRAME_M1&initial_date={initial_date}%2000%3A00%3A00.001&final_date={final_date}%2023%3A59%3A59.999")
print(r.status_code)
json_data = r.json()

200


In [3]:
df = pd.DataFrame(list(json_data.values()))
df.rename(columns = {'open':'Open', 'high':'High', 'low':'Low', 'close':'Close', 'real_volume':'Volume', 'date': 'Date'}, inplace = True)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index(['Date'], inplace=True)
df = df.loc[:,['Open', 'High', 'Low', 'Close', 'Volume']]
df

,Open,High,Low,Close,Volume
Date,,,,,
2022-08-01 09:00:00,103825.0,103830.0,103640.0,103670.0,21793
2022-08-01 09:01:00,103670.0,103820.0,103645.0,103785.0,52268
2022-08-01 09:02:00,103790.0,103830.0,103740.0,103765.0,21140
2022-08-01 09:03:00,103760.0,103775.0,103695.0,103710.0,16715
2022-08-01 09:04:00,103705.0,103715.0,103670.0,103675.0,11696
...,...,...,...,...,...
2022-08-01 17:50:00,102710.0,102740.0,102705.0,102710.0,5834
2022-08-01 17:51:00,102710.0,102720.0,102690.0,102700.0,2706
2022-08-01 17:52:00,102700.0,102730.0,102700.0,102715.0,3368


In [25]:
macd, macdsignal, macdhist = talib.MACD(df.Close, fastperiod=5, slowperiod=15, signalperiod=10)
macd_changing_trend = analysis.cross_by_value(macdhist, 0)
macd_changing_trend_up = analysis.crossover_by_value(macdhist, 0)
macd_changing_trend_down = analysis.crossbelow_by_value(macdhist, 0)

close_macd_changing_trend = df.Close[macd_changing_trend]
close_macd_changing_trend_up = df.Close[macd_changing_trend_up]
close_macd_changing_trend_down = df.Close[macd_changing_trend_down]
macd_hist_changing_trend = macdhist[macd_changing_trend]

adx = talib.ADX(df.High, df.Low, df.Close, timeperiod=5)
adx_trend = adx > 25
macd_trend_up_confirmation = macd_changing_trend_up & adx_trend
macd_trend_down_confirmation = macd_changing_trend_down & adx_trend

close_macd_changing_trend_up = df.Close[macd_trend_up_confirmation]
close_macd_changing_trend_down = df.Close[macd_trend_down_confirmation]

# slope_close = talib.LINEARREG_SLOPE(df.Close, timeperiod=5)
slope_volume = talib.LINEARREG_SLOPE(df.Volume, timeperiod=5)
# angle_close = talib.LINEARREG_ANGLE(df.Close, timeperiod=5)
angle_volume = talib.LINEARREG_ANGLE(df.Volume, timeperiod=5)
# divergences_slopes = ((angle_close > 0) & (angle_volume < 0)) | ((angle_close < 0) & (angle_volume > 0))
# close_divergences_slopes = df.Close[divergences_slopes]

In [24]:
fig = make_subplots(rows=4, cols=1)

fig.add_trace(go.Scatter(
    x=df.index, y=df.Close, name='Close Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=close_macd_changing_trend_up.index, y=close_macd_changing_trend_up, name='Changing trend up', mode="markers"
), row=1, col=1)


fig.add_trace(go.Scatter(
    x=close_macd_changing_trend_down.index, y=close_macd_changing_trend_down, name='Changing trend down', mode="markers"
), row=1, col=1)

fig.add_trace(go.Bar(
    x=macdhist.index, y=macdhist, name='MACD Histogram'
), row=2, col=1)

# fig.add_trace(go.Bar(
#     x=df.index, y=macdhist, name='MACD Histogram'
# ), row=2, col=1)


fig.add_trace(go.Scatter(
    x=adx.index, y=adx, name='ADX'
), row=3, col=1)

fig.add_trace(go.Scatter(
        x = [df.index.min(), df.index.max()],
        y = [20, 20],
        mode = "lines",
        name = "ADX line trend low",
        marker = dict(color = 'rgba(255, 0, 0, 0.8)')
    ), row=3, col=1)

fig.add_trace(go.Scatter(
        x = [df.index.min(), df.index.max()],
        y = [25, 25],
        mode = "lines",
        name = "ADX line trend increasing",
        marker = dict(color = 'rgba(22, 105, 55, 0.8)')
    ), row=3, col=1)


fig.add_trace(go.Scatter(
    x=angle_volume.index, y=angle_volume, name='Angle Volume'
), row=4, col=1)
